In [93]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
    
import numpy as np
import pandas as pd
import pylab as plt

from pyace import *
from tensorpotential.potentials.ace import ACE
from tensorpotential.tensorpot import TensorPotential
from tensorpotential.fit import FitTensorPotential
from tensorpotential.constants import *
from tensorpotential.utils.utilities import generate_tp_atoms
import tensorflow as tf

from ase import Atoms
from ase.calculators.singlepoint import SinglePointCalculator

The first step to constructing an ACE model is by providing 4 main components (aka basis shape). 
- Just set `deltaSplineBins` to `0.001` (not sure what this is) 
- We set the embeddings for each element. 
    - We are working with the Finnis-Sinclair model, so : `'npot': 'FinnisSinclairShiftedScaled'` 
    - We have two copies of the linear ACE model, so : `'ndensity': 2` 
    - We want to set the parameters $[c_1, c_2, c_3, c_4]$ of the embedding function $f \mapsto c_1 \, f^{c_2} + c_3 \, f^{c_4}$ to $[1, 1, 1, 0.5]$ so that it matches our embedding function $ f \mapsto f + \sqrt{f}$ 
- We set the bonds for each possible pairs of elements. 
    - The radial base is set as `'radbase': 'SBessel'` as the Bessel function. and its parameters `'radparameters': [2.0]`. 
    - The outer cutoff is specified by two parameters `'rcut': 5.0, 'dcut': 0.01`, which is applied in a range `[rcut - dcut, rcut]`. 
    - The inner cutoff is specified by two parameters `'r_in': 1.0, 'delta_in': 0.5`, which is applied in a range `[r_in, r_in + delta_in]`. 
    - Core repulsion (?) 
- The functions specify the order and max degree of the elements. 
    - df

In [81]:
cutoff = 3

potential_config = {
        # Step 0. define deltaSplineBins
        'deltaSplineBins': 0.001,

        # Step 1. specify all elements of the basis
        'elements': ['Cu'],

        # Step 2. specify embeddings for all elements, using 'ALL' or elements name keywords
        'embeddings': {'ALL': {
                'ndensity': 2,
                'fs_parameters': [1, 1, 1, 0.5],                ## non-linear embedding function: 1*rho_1^1 + 1*rho_2^0.5
                'npot': 'FinnisSinclairShiftedScaled',

                #'rho_core_cut': 2000000.,
                #'drho_core_cut': 250,
        },
        },
        # Step 3. specify bonds for all elements, using 'ALL', UNARY, BINARY or elements pairs
        'bonds': {'ALL': {
                'radbase': 'SBessel',
                'radparameters': [2.0],
                'rcut': cutoff,
                'dcut': 0.01,
                #'NameOfCutoffFunction': 'cos',
                #'core-repulsion': [0.0, 1.0],
        },
        },

        # Step 4. Specify BBasisFunctions list for each block using ALL, UNARY, BINARY, ..., QUINARY keywords
        # setup per-rank nradmax_by_orders and lmax_by_orders
        'functions': {
                'ALL': {
                        'nradmax_by_orders': [1, 1],
                        'lmax_by_orders': [1, 1]
                },
        }
}

After this, we create the basis by inputting the `potential_config` data. The `func_coefs_initializer` can be set to `"zero"` to initialize to 0s or `"random"` for random initialization. However, the first two parameters will always be 1, since they are the constant functions (?). 

In [82]:
np.random.seed(42)

bbasisconf = create_multispecies_basis_config(potential_config, func_coefs_initializer="zero")

Now we just create the ACE model. There are many ways to initalize an `ACE` object, but we do it using a basis configuration. 

Some useful commands: 
- `ace.get_coefs()`: returns a tensor of the parameters. 
- `ace.set_coefs(c)`: sets the coefficients to whatever parameters defined by `c`
- `ace.lmax` : returns the order(?) 
- `ace.rankmax` : returns the max degree(?) 
- `ace.rcut` : returns the cutoff radius 
- `ace.nradmax` : 

In [83]:
ace = ACE(bbasisconf)

Let's grab an `Atoms` object from our dataset. 

In [85]:
df = pd.read_pickle('../Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip', compression="gzip")
row = df.iloc[2]
at = row['ase_atoms']

t = generate_tp_atoms(at, cutoff=cutoff)
print(t)

{'_ind_i': array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
         1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
         2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
         4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,
         5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,
         7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
         9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
        11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12,
        12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14,
        14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15,
        15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
        17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 18, 18, 18, 18, 18,
  

(HELP) Now we need some way to compute the atomic energy. We can do this by calling `compute_atomic_energy(self, r_ij, ind_i, mu_i, mu_j, ind_j)`, but I am not sure what the parameters are. 

In [105]:
r_ij = t['_positions']
ind_i = t['_ind_i']
ind_j = t['_ind_j']
mu_i = t['_mu_i']
mu_j = t['_mu_j']

In [107]:
ace = ACE(bbasisconf)

ace.compute_atomic_energy(r_ij, ind_i, mu_i, mu_j, ind_j)

InvalidArgumentError: data.shape must start with partitions.shape, got data.shape = [64,1], partitions.shape = [768] [Op:DynamicPartition]

Now we wrap it into a `TensorPotential` object with information about the cost function. 

In [5]:
tp = TensorPotential(ace, loss_specs={
        LOSS_TYPE: 'per-atom',
        LOSS_FORCE_FACTOR: 1,
        LOSS_ENERGY_FACTOR: 10,
        L1_REG: 0e-8,
        L2_REG: 0e-8
    })

In [6]:
tpf = FitTensorPotential(tp, eager=False)

Now we can finally fit the model with the dataframe containing the prepared data. 

In [ ]:
df = pd.DataFrame()
tpf.fit(df, niter=10, batch_size=105, optimizer='BFGS')